In [1]:
import numpy as np 
import pandas as pd
import torch 

import sys
import os
import pickle 
current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

from plotting.TS_analysis import plot_TS
from load_inputs.weather import load_weather_data
from constants.paths import FOLDER_PATH

/Users/romainrochas/opt/anaconda3/envs/pytorch-2.0.1/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Weather Aggregated by 30 min 

In [2]:
if torch.cuda.is_available():
    folder_path = f"{FOLDER_PATH}/weather"
else:
    FOLDER_PATH = '../../../../Data'
    folder_path = f"{FOLDER_PATH}/Meteo"
csv_path = 'donnees-meteo.csv'

df_weather = load_weather_data(folder_path,csv_path)
df_weather.head()

,id_station,date,temperature,precip,duree_prec,wind_ms,solar
35088,69029001,2019-01-01 00:00:00,"6,8",0,0.0,"3,6",0.0
35089,69029001,2019-01-01 01:00:00,"6,7",0,0.0,"3,3",0.0
35090,69029001,2019-01-01 02:00:00,"6,7",0,0.0,"1,5",0.0
35091,69029001,2019-01-01 03:00:00,"6,8",0,0.0,"3,2",0.0
35092,69029001,2019-01-01 04:00:00,"6,7",0,0.0,"3,4",0.0


# Weather aggregated by 10 min from Infoclimat API :

In [3]:
start = '01-01-2019'
end = '01-01-2021'
reindex = pd.date_range(start,end, freq= '1h')[:-1]

csv_path = 'df_weather_infoclimat.csv'
df_weather_api = pd.read_csv(f"{folder_path}/{csv_path}",index_col=0)
df_rain_api = df_weather_api.pivot_table(index = 'datetime',columns = 'station_id',values = 'pluie_intensite')
df_rain_api.index = pd.to_datetime(df_rain_api.index)
df_rain_api_verif = df_rain_api.resample('1h').mean()
df_rain_api_verif = df_rain_api_verif.reindex(reindex)


df_weather_copy = df_weather.copy()
df_weather_copy.date = pd.to_datetime(df_weather_copy.date)
df_weather_copy.drop_duplicates(subset= ['id_station','date'])
df_weather_copy['precip'] =df_weather_copy['precip'].str.replace(',','.').astype(float)
df_weather_copy = df_weather_copy.pivot_table(index = 'date',columns = 'id_station',values = 'precip')
df_weather_copy = df_weather_copy.reindex(reindex)


df_merged = pd.concat([df_rain_api_verif,df_weather_copy],axis=1)

from plotting.TS_analysis import plot_TS
plot_TS(df_merged,width = 1000,height = 400,title = 'Rain Intensity',bool_show = True)

FileNotFoundError: [Errno 2] No such file or directory: '../../../../Data/Meteo/df_weather_infoclimat.csv'